In [86]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from scipy.stats import gumbel_r

# Load your dataset
df = pd.read_csv("./data/landslide_data_final.csv")

timeframes = ["10yr", "20yr", "50yr"]

# Calculate probabilities from Gumbel distribution
loc = 10  # Location parameter (adjust if needed)
scale = 5  # Scale parameter (adjust if needed)
return_periods = [10, 20, 50]  # Corresponding to your timeframes
probabilities = [1 - (1 / rp) for rp in return_periods]
weights = gumbel_r.pdf(return_periods, loc=loc, scale=scale) * probabilities
weights /= weights.sum()  # Normalize weights

# Invert the weights to give higher weight to rarer events
weights = 1 / weights
weights /= weights.sum()  # Re-normalize weights

# Create a dictionary mapping timeframes to weights
weights_dict = dict(zip(timeframes, weights))

for timeframe in timeframes:
    df[f"Flow_accumulation_{timeframe}_multiplied_log"] *= weights_dict[timeframe]
    df[f"Runoff_{timeframe}"] *= weights_dict[timeframe]

    X = df[
        [
            f"Flow_accumulation_{timeframe}_multiplied_log",
            "Slope_classification",
            "Flow_direction",
            f"Runoff_{timeframe}",
            "Fill_sinks",
            "Basins",
            "intersects_buildings",
            "GLG",
        ]
    ]
    y = df["Landslide_Occurred"]

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Preprocessing pipeline
    numeric_features = [
        f"Flow_accumulation_{timeframe}_multiplied_log",
        f"Runoff_{timeframe}",
        "Flow_direction",
        "Fill_sinks",
        "Basins",
        "Slope_classification",
        "intersects_buildings",
    ]
    categorical_features = ["GLG"]

    numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

    categorical_transformer = Pipeline(
        steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))]
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ]
    )

    # Fit and transform the training data
    X_train_transformed = preprocessor.fit_transform(X_train)

    # Transform the test data
    X_test_transformed = preprocessor.transform(X_test)

    # Create and train the model with hyperparameter tuning
    model = LogisticRegression()

    param_grid = {
        "C": [0.1, 1, 10],
        "penalty": ["l1", "l2"],
        "solver": ["liblinear", "saga"],
    }

    grid_search = GridSearchCV(model, param_grid, cv=5, scoring="roc_auc")
    grid_search.fit(X_train_transformed, y_train)

    best_model = grid_search.best_estimator_

    # Make predictions on the test set
    y_pred = best_model.predict(X_test_transformed)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for {timeframe}: {accuracy:.4f}")

    # Make predictions for the entire dataset
    df[f"Susceptibility_{timeframe}"] = best_model.predict_proba(
        preprocessor.transform(X)
    )[:, 1]

df["Combined_Susceptibility"] = df[
    [f"Susceptibility_{timeframe}" for timeframe in timeframes]
].mean(axis=1)  # Average the individual susceptibilities

# Classify into risk levels based on quantiles of the combined susceptibility
low_threshold = df["Combined_Susceptibility"].quantile(0.1)
high_threshold = df["Combined_Susceptibility"].quantile(0.9)

for timeframe in timeframes:
    df_timeframe = df[
        [
            "Landslide_ID",
            "Latitude",
            "Longitude",
            f"Susceptibility_{timeframe}",
            "Combined_Susceptibility",  # Include for combined risk assessment
        ]
    ].copy()

    # Classify into risk levels using combined susceptibility thresholds
    df_timeframe["Risk_Level"] = "Medium"
    df_timeframe.loc[
        df_timeframe["Combined_Susceptibility"] < low_threshold, "Risk_Level"
    ] = "Low"
    df_timeframe.loc[
        df_timeframe["Combined_Susceptibility"] > high_threshold, "Risk_Level"
    ] = "High"

    # Select only the required columns and IDs
    df_timeframe = df_timeframe[
        (df_timeframe["Landslide_ID"] >= 0) & (df_timeframe["Landslide_ID"] <= 152)
    ][
        [
            "Landslide_ID",
            "Latitude",
            "Longitude",
            f"Susceptibility_{timeframe}",
            "Risk_Level",
        ]
    ].sort_values(by="Landslide_ID")

    # Count risk level values
    risk_counts = df_timeframe["Risk_Level"].value_counts()
    print(f"Risk level counts for {timeframe}:\n{risk_counts}")

    df_timeframe.to_csv(f"./data/landslide_susceptibility_{timeframe}.csv", index=False)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was

Accuracy for 10yr: 0.8225


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was

Accuracy for 20yr: 0.8225
Accuracy for 50yr: 0.8225
Risk level counts for 10yr:
Risk_Level
Medium    107
High       40
Low         4
Name: count, dtype: int64
Risk level counts for 20yr:
Risk_Level
Medium    107
High       40
Low         4
Name: count, dtype: int64
Risk level counts for 50yr:
Risk_Level
Medium    107
High       40
Low         4
Name: count, dtype: int64


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was

# Random Forest

In [90]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import gumbel_r, randint, uniform

# Load your dataset
df = pd.read_csv("./data/landslide_data_final.csv")

timeframes = ["10yr", "20yr", "50yr"]

# Calculate probabilities from Gumbel distribution
loc = 10  # Location parameter (adjust if needed)
scale = 5  # Scale parameter (adjust if needed)
return_periods = [10, 20, 50]  # Corresponding to your timeframes
probabilities = [1 - (1 / rp) for rp in return_periods]
weights = gumbel_r.pdf(return_periods, loc=loc, scale=scale) * probabilities
weights /= weights.sum()  # Normalize weights

# Invert the weights to give higher weight to rarer events
weights = 1 / weights
weights /= weights.sum()  # Re-normalize weights

# Create a dictionary mapping timeframes to weights
weights_dict = dict(zip(timeframes, weights))

for timeframe in timeframes:
    # Scale flow accumulation and runoff based on exponential weights
    df[f"Flow_accumulation_{timeframe}_multiplied_log"] *= weights_dict[timeframe]
    df[f"Runoff_{timeframe}"] *= weights_dict[timeframe]

    X = df[
        [
            f"Flow_accumulation_{timeframe}_multiplied_log",
            "Slope_classification",
            "Flow_direction",
            f"Runoff_{timeframe}",
            "Fill_sinks",
            "Basins",
            "intersects_buildings",
            "GLG",
        ]
    ]
    y = df["Landslide_Occurred"]

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Preprocessing pipeline
    numeric_features = [
        f"Flow_accumulation_{timeframe}_multiplied_log",
        f"Runoff_{timeframe}",
        "Flow_direction",
        "Fill_sinks",
        "Basins",
    ]
    categorical_features = ["Slope_classification", "GLG"]

    numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

    categorical_transformer = Pipeline(
        steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))]
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ]
    )

    # Fit and transform the training data
    X_train_transformed = preprocessor.fit_transform(X_train)

    # Transform the test data
    X_test_transformed = preprocessor.transform(X_test)

    # Create and train the model with hyperparameter tuning
    model = RandomForestClassifier()

    param_grid = {
        "n_estimators": randint(50, 200),  # Number of trees in random forest
        "max_depth": randint(3, 15),  # Maximum number of levels in tree
        "min_samples_split": randint(2, 20),  # Minimum number of samples required to split a node
        "min_samples_leaf": randint(1, 10),  # Minimum number of samples required at each leaf node
        "max_features": uniform(0.1, 0.9),  # Number of features to consider at every split
        "bootstrap": [True, False],  # Method of selecting samples for training each tree
    }


    grid_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=50,  # Number of parameter settings that are sampled
        cv=5,
        scoring="roc_auc",
        random_state=42,
        n_jobs=-1,  # Use all available cores
    )
    grid_search.fit(X_train_transformed, y_train)

    best_model = grid_search.best_estimator_
    # Make predictions
    y_pred = best_model.predict(X_test_transformed)
    y_prob = best_model.predict_proba(X_test_transformed)[:, 1]

    # Evaluate the model
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("AUC:", roc_auc_score(y_test, y_prob))

    # Make predictions for the entire dataset
    df[f"Susceptibility_{timeframe}"] = best_model.predict_proba(
        preprocessor.transform(X)
    )[:, 1]

df["Combined_Susceptibility"] = df[
    [f"Susceptibility_{timeframe}" for timeframe in timeframes]
].mean(axis=1)  # Average the individual susceptibilities

# Classify into risk levels based on quantiles of the combined susceptibility
low_threshold = df["Combined_Susceptibility"].quantile(0.33)
high_threshold = df["Combined_Susceptibility"].quantile(0.67)

for timeframe in timeframes:
    df_timeframe = df[
        [
            "Landslide_ID",
            "Latitude",
            "Longitude",
            f"Susceptibility_{timeframe}",
            "Combined_Susceptibility",  # Include for combined risk assessment
        ]
    ].copy()

    # Classify into risk levels using combined susceptibility thresholds
    df_timeframe["Risk_Level"] = "Medium"
    df_timeframe.loc[
        df_timeframe["Combined_Susceptibility"] < low_threshold, "Risk_Level"
    ] = "Low"
    df_timeframe.loc[
        df_timeframe["Combined_Susceptibility"] > high_threshold, "Risk_Level"
    ] = "High"

    # Select only the required columns and IDs
    df_timeframe = df_timeframe[
        (df_timeframe["Landslide_ID"] >= 0) & (df_timeframe["Landslide_ID"] <= 400)
    ][
        [
            "Landslide_ID",
            "Latitude",
            "Longitude",
            f"Susceptibility_{timeframe}",
            "Risk_Level",
        ]
    ].sort_values(by="Landslide_ID")

    # Count risk level values
    risk_counts = df_timeframe["Risk_Level"].value_counts()
    print(f"Risk level counts for {timeframe}:\n{risk_counts}")

    df_timeframe.to_csv(f"./data/landslide_susceptibility_{timeframe}.csv", index=False)

Accuracy: 0.8165680473372781
              precision    recall  f1-score   support

           0       0.83      0.99      0.90       139
           1       0.33      0.03      0.06        30

    accuracy                           0.82       169
   macro avg       0.58      0.51      0.48       169
weighted avg       0.74      0.82      0.75       169

Confusion Matrix:
 [[137   2]
 [ 29   1]]
AUC: 0.7046762589928058
Accuracy: 0.8106508875739645
              precision    recall  f1-score   support

           0       0.82      0.98      0.89       139
           1       0.25      0.03      0.06        30

    accuracy                           0.81       169
   macro avg       0.54      0.51      0.48       169
weighted avg       0.72      0.81      0.75       169

Confusion Matrix:
 [[136   3]
 [ 29   1]]
AUC: 0.7182254196642684
Accuracy: 0.8224852071005917
              precision    recall  f1-score   support

           0       0.83      0.99      0.90       139
           1      

In [93]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Load your dataset
df = pd.read_csv("./data/landslide_data_final.csv")

# Calculate the average flow accumulation and runoff across timeframes
df["Flow_accumulation_avg"] = df[
    [f"Flow_accumulation_{timeframe}_multiplied_log" for timeframe in ["10yr", "20yr", "50yr"]]
].mean(axis=1)
df["Runoff_avg"] = df[[f"Runoff_{timeframe}" for timeframe in ["10yr", "20yr", "50yr"]]].mean(axis=1)

# Prepare the feature matrix and target variable
X = df[
    [
        "Flow_accumulation_avg",
        "Slope_classification",
        "Flow_direction",
        "Runoff_avg",
        "Fill_sinks",
        "Basins",
        "intersects_buildings",
        "GLG",
    ]
]
y = df["Landslide_Occurred"]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Preprocessing pipeline
numeric_features = [
    "Flow_accumulation_avg",
    "Runoff_avg",
    "Flow_direction",
    "Fill_sinks",
    "Basins",
    "Slope_classification",
    "intersects_buildings",
]
categorical_features = ["GLG"]

numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

categorical_transformer = Pipeline(
    steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# Fit and transform the training data
X_train_transformed = preprocessor.fit_transform(X_train)

# Transform the test data
X_test_transformed = preprocessor.transform(X_test)

# Create and train the model with hyperparameter tuning
model = LogisticRegression()

param_grid = {
    "C": [0.1, 1, 10],
    "penalty": ["l1", "l2"],
    "solver": ["liblinear", "saga"],
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring="roc_auc")
grid_search.fit(X_train_transformed, y_train)

best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test_transformed)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Make predictions for the entire dataset
df["Susceptibility"] = best_model.predict_proba(preprocessor.transform(X))[:, 1]

# Classify into risk levels based on quantiles of the susceptibility
low_threshold = df["Susceptibility"].quantile(0.2)
high_threshold = df["Susceptibility"].quantile(0.8)

df["Risk_Level"] = "Medium"
df.loc[df["Susceptibility"] < low_threshold, "Risk_Level"] = "Low"
df.loc[df["Susceptibility"] > high_threshold, "Risk_Level"] = "High"

# Select only the required columns and IDs
df_output = df[
    (df["Landslide_ID"] >= 0) & (df["Landslide_ID"] <= 152)
][
    [
        "Landslide_ID",
        "Latitude",
        "Longitude",
        "Susceptibility",
        "Risk_Level",
    ]
].sort_values(by="Landslide_ID")

# Count risk level values
risk_counts = df_output["Risk_Level"].value_counts()
print(f"Risk level counts:\n{risk_counts}")

df_output.to_csv(f"./data/landslide_susceptibility.csv", index=False)

Accuracy: 0.8225
Risk level counts:
Risk_Level
Medium    84
High      56
Low       11
Name: count, dtype: int64


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was

Minimize  $E[Z] = \sum_{(i,j) \in P} \frac{D_{ij}}{S_{ij} \cdot (1 - \max(CR_{ij,flood}(t), CR_{ij,landslide} \cdot W_{flood-landslide}(t)))}$


Minimize  $E[Z] = \sum_{(i,j) \in P} \frac{D_{ij}}{S_{ij} \cdot (1 - \max(CR_{ij,flood_{map\_s}}, CR_{ij,landslide} \cdot W_{flood-landslide_{map\_s}}))}$